<a href="https://colab.research.google.com/github/pasmi369/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [450 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 

In [3]:
#Start Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Vine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [35]:
# Load in data from S3 bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show(20,truncate=False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                                                              |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                     

In [38]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|RTIS3L2M1F5SM |5          |0            |0          |N   |Y                |
|R1ZV7R40OLHKD |5          |0            |0          |N   |Y                |
|R3BH071QLH8QMC|1          |0            |1          |N   |Y                |
|R127K9NTSXA2YH|3          |0            |0          |N   |Y                |
|R32ZWUXDJPW27Q|4          |0            |0          |N   |Y                |
|R3AQQ4YUKJWBA6|1          |0            |0          |N   |Y                |
|R2F0POU5K6F73F|5          |0            |0          |N   |Y                |
|R3VNR804HYSMR6|5          |0            |0          |N   |Y                |
|R3GZTM72WA2QH |5          |0            |0          |N   |Y                |
|RNQOY62705W1K |4          |0            |0          |N   |Y    

In [39]:
# filter for only columns with 20 or more total votes
clean_vine_df = vine_df.filter(vine_df["total_votes"]>=20)
clean_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R4PKAZRQJJX14 |1          |21           |34         |N   |N                |
|R2CI0Y288CC7E2|1          |21           |35         |N   |Y                |
|R127WEQY2FM1T3|1          |147          |175        |N   |Y                |
|R3EZ0EPYLDA34S|1          |14           |31         |N   |Y                |
|R2FJ94555FZH32|2          |55           |60         |N   |N                |
|R1U3AR67RE273L|1          |51           |65         |N   |Y                |
|R3PZOXA5X1U8KW|4          |31           |36         |N   |N                |
|R6KTC1OPIOIIG |2          |19           |34         |N   |Y                |
|R36O341WWXXKNP|5          |28           |31         |N   |N                |
|R3GSK9MM8DNOYI|1          |4            |32         |N   |N    

In [40]:
# filter for only columns with mostly helpful votes
helpful_vine_df = clean_vine_df.filter("helpful_votes/total_votes>=0.5")
helpful_vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R4PKAZRQJJX14 |1          |21           |34         |N   |N                |
|R2CI0Y288CC7E2|1          |21           |35         |N   |Y                |
|R127WEQY2FM1T3|1          |147          |175        |N   |Y                |
|R2FJ94555FZH32|2          |55           |60         |N   |N                |
|R1U3AR67RE273L|1          |51           |65         |N   |Y                |
|R3PZOXA5X1U8KW|4          |31           |36         |N   |N                |
|R6KTC1OPIOIIG |2          |19           |34         |N   |Y                |
|R36O341WWXXKNP|5          |28           |31         |N   |N                |
|R10LZVBLQHBVJ0|2          |151          |198        |N   |N                |
|R1VR5GLGY1GE7N|1          |49           |51         |N   |Y    

In [41]:
# load in a sql function to use columns 
from pyspark.sql.functions import col

# Filter new DF that retrieves all rows where review for Vine(paid) column with vine == "Y"
helpful_vine_paid_df = helpful_vine_df.filter(col("vine") == 'Y')
helpful_vine_paid_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|5          |56           |63         |Y   |N                |
|R10FO5UKKVZBK2|3          |23           |23         |Y   |N                |
|RM4KSGEOR7MU1 |5          |19           |24         |Y   |N                |
|RG7VRMYLEXD23 |4          |22           |26         |Y   |N                |
|R11O4YSCPSNL6L|3          |20           |26         |Y   |N                |
|R286MFBAJ8NPD6|5          |46           |51         |Y   |N                |
|R1JRR530H4COA2|5          |22           |28         |Y   |N                |
|RQ5WD90PUNBU9 |5          |21           |24         |Y   |N                |
|R12648VHCQWUV9|4          |21           |28         |Y   |N                |
|R3KAW29CJ8L6DQ|5          |17           |20         |Y   |N    

In [55]:
# Filter new DF that retrieves all rows where review for Vine(unpaid) column with vine == "N"

helpful_vine_unpaid_df = helpful_vine_df.filter(col("vine") == 'N')
helpful_vine_unpaid_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R4PKAZRQJJX14 |1          |21           |34         |N   |N                |
|R2CI0Y288CC7E2|1          |21           |35         |N   |Y                |
|R127WEQY2FM1T3|1          |147          |175        |N   |Y                |
|R2FJ94555FZH32|2          |55           |60         |N   |N                |
|R1U3AR67RE273L|1          |51           |65         |N   |Y                |
|R3PZOXA5X1U8KW|4          |31           |36         |N   |N                |
|R6KTC1OPIOIIG |2          |19           |34         |N   |Y                |
|R36O341WWXXKNP|5          |28           |31         |N   |N                |
|R10LZVBLQHBVJ0|2          |151          |198        |N   |N                |
|R1VR5GLGY1GE7N|1          |49           |51         |N   |Y    

In [43]:
# Determine total number of reviews
helpful_vine_df.count()

40565

In [44]:
# Determine total number of paid reviews
helpful_vine_paid_df.count()

94

In [56]:
# Determine total number of unpaid reviews
helpful_vine_unpaid_df.count()

40471

In [46]:
# Determine number of 5-star reviews
helpful_vine_df.filter(col("star_rating") == 5).count()

15711

In [47]:
# Determine number of paid 5-star reviews
helpful_vine_paid_df.filter(col("star_rating") == 5).count()

48

In [57]:
# Determine number of unpaid 5-star reviews
helpful_vine_unpaid_df.filter(col("star_rating") == 5).count()

15663

In [52]:
#Determine percentage of 5-star paid reviews
a = helpful_vine_paid_df.count()
b = helpful_vine_paid_df.filter(col("star_rating") == 5).count()
percentage_paid = b/a*100
print(str(round(percentage_paid,2)) + "% of paid reviews are 5-stars")


51.06% of paid reviews are 5-stars


In [58]:
# Determine percentage of 5-star unpaid reviews
c = helpful_vine_unpaid_df.count()
d = helpful_vine_unpaid_df.filter(col("star_rating") == 5).count()
percentage_unpaid = d/c * 100
print(str(round(percentage_unpaid,2)) + "% of unpaid reviews are 5-stars")

38.7% of unpaid reviews are 5-stars
